In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


/Users/aayeshaaayesha/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
import pandas as pd
df_low_sample = pd.read_excel('person_low_sample.xlsx')
df_med_sample = pd.read_excel('person_med_sample.xlsx')
df_good_sample = pd.read_excel('person_good_sample.xlsx')


In [5]:
from statistics import mean 
col_list = ['accomp1p', 'accomp2p', 'accomp3p']
def sentiForCol(col_list):
    pos_mean_ls = []
    for k in range(df_low_sample.shape[0]):
        pos_ls = []
        for j in range(0, len(col_list)):
          text = df_low_sample.iloc[k][col_list[j]]
          text = preprocess(text)
          encoded_input = tokenizer(text, return_tensors='pt')
          output = model(**encoded_input)
          scores = output[0][0].detach().numpy()
          scores = softmax(scores)
          ranking = np.argsort(scores)
          ranking = ranking[::-1]
          for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
        pos_mean = mean(pos_ls)
        pos_mean_ls.append(pos_mean)
    return pos_mean_ls
print(sentiForCol(col_list))
low = sentiForCol(col_list)

[0.16416183, 0.63723654, 0.31162542, 0.3023177, 0.025856895, 0.38602617, 0.20151478, 0.6348201, 0.5300344]


In [6]:
col_list = ['accomp1p', 'accomp2p', 'accomp3p']
def sentiForCol1(col_list):
    pos_mean_ls = []
    for k in range(df_med_sample.shape[0]):
        pos_ls = []
        for j in range(0, len(col_list)):
          text = df_med_sample.iloc[k][col_list[j]]
          text = preprocess(text)
          encoded_input = tokenizer(text, return_tensors='pt')
          output = model(**encoded_input)
          scores = output[0][0].detach().numpy()
          scores = softmax(scores)
          ranking = np.argsort(scores)
          ranking = ranking[::-1]
          for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
        pos_mean = mean(pos_ls)
        pos_mean_ls.append(pos_mean)
    return pos_mean_ls
print(sentiForCol1(col_list))
med = sentiForCol1(col_list)

[0.89269495, 0.33875483, 0.5117333, 0.42994776, 0.28494498, 0.4347854, 0.45608547, 0.022548223, 0.31565517]


In [7]:
col_list = ['accomp1p', 'accomp2p', 'accomp3p']
def sentiForCol1(col_list):
    pos_mean_ls = []
    for k in range(df_good_sample.shape[0]):
        pos_ls = []
        for j in range(0, len(col_list)):
          text = df_good_sample.iloc[k][col_list[j]]
          text = preprocess(text)
          encoded_input = tokenizer(text, return_tensors='pt')
          output = model(**encoded_input)
          scores = output[0][0].detach().numpy()
          scores = softmax(scores)
          ranking = np.argsort(scores)
          ranking = ranking[::-1]
          for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if l=='positive':
                pos_ls.append(s)
        pos_mean = mean(pos_ls)
        pos_mean_ls.append(pos_mean)
    return pos_mean_ls
print(sentiForCol1(col_list))
good = sentiForCol1(col_list)

[0.7515275, 0.40131566, 0.23390968, 0.9166862, 0.3598943, 0.96758866, 0.6645674, 0.5710844, 0.8281719]


In [8]:
df_low_interface_t_test = pd.DataFrame({'experience': 'low', 'interface_pos': low })
df_med_interface_t_test = pd.DataFrame({'experience': 'med', 'interface_pos': med })
df_good_interface_t_test = pd.DataFrame({'experience': 'good', 'interface_pos': good })


In [9]:
df_experience_combine_interface_t_test = pd.concat([df_low_interface_t_test, df_med_interface_t_test, df_good_interface_t_test], axis=0)
df_experience_combine_interface_t_test

,experience,interface_pos
0,low,0.164162
1,low,0.637237
2,low,0.311625
3,low,0.302318
4,low,0.025857
5,low,0.386026
6,low,0.201515
7,low,0.634820
8,low,0.530034
0,med,0.892695


In [58]:
from scipy import stats
import numpy as np


interface_pos_low = df_experience_combine_interface_t_test[df_experience_combine_interface_t_test['experience'] == 'low']['interface_pos']
interface_pos_med = df_experience_combine_interface_t_test[df_experience_combine_interface_t_test['experience'] == 'med']['interface_pos']
interface_pos_good = df_experience_combine_interface_t_test[df_experience_combine_interface_t_test['experience'] == 'good']['interface_pos']

# Perform the independent t-test
t_stat, p_value = stats.ttest_ind(interface_pos_low, interface_pos_med)

print(f"T-statistic: {t_stat}")
print(f"P-value: {p_value}")

# Interpretation
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis - there is a significant difference in interface experience between age groups.")
else:
    print("Fail to reject the null hypothesis - there is no significant difference in interface experience between age groups.")

T-statistic: -0.5239547316411325
P-value: 0.6074924147228514
Fail to reject the null hypothesis - there is no significant difference in interface experience between age groups.


In [10]:
from scipy import stats
from itertools import combinations
hobbies = df_experience_combine_interface_t_test['experience'].unique()

# Perform pairwise t-tests
pairwise_results = []
for (hobby1, hobby2) in combinations(hobbies, 2):
    group1 = df_experience_combine_interface_t_test[df_experience_combine_interface_t_test['experience'] == hobby1]['interface_pos']
    group2 = df_experience_combine_interface_t_test[df_experience_combine_interface_t_test['experience'] == hobby2]['interface_pos']
    t_stat, p_value = stats.ttest_ind(group1, group2)
    pairwise_results.append(((hobby1, hobby2), t_stat, p_value))

# Print results
for (hobby1, hobby2), t_stat, p_value in pairwise_results:
    print(f"{hobby1} vs. {hobby2}: T-statistic = {t_stat:.2f}, P-value = {p_value:.4f}")


low vs. med: T-statistic = -0.52, P-value = 0.6075
low vs. good: T-statistic = -2.49, P-value = 0.0243
med vs. good: T-statistic = -1.93, P-value = 0.0717
